In [180]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append('../')
from pulpo import pulpo
import pandas as pd
pd.set_option('display.max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup the Worker

In [181]:
project = "ammonia_reduced"
databases = ["ecoinvent-3.10-cutoff", "ammonia-reduced"]
methods = "('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')"

In [182]:
# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
# GAMS_PATH = r"C:\APPS\GAMS\win64\40.1\gams.exe"

In [183]:
pulpo_worker = pulpo.PulpoOptimizer(project, databases, methods, directory)
pulpo_worker.intervention_matrix="ecoinvent-3.10-biosphere"

In [184]:
pulpo_worker.get_lci_data()

### Get the Choices

In [185]:

choices_biomethane_CS = ["biogas upgrading to biomethane, chemical scrubbing"]
choices_biomethane_CSwCCS = ["biogas upgrading to biomethane, chemical scrubbing w/ CCS"]
choices_biomethane_WS = ["biogas upgrading to biomethane, water scrubbing"]
choices_biomethane_WSwCCS = ["biogas upgrading to biomethane, water scrubbing w/ CCS"]

choices_methane_market = [
    "market for bio methane",
    "market group for natural gas, high pressure"
]

choices_hydrogen_SMR = ["hydrogen production, steam methane reforming fg"]
choices_hydrogen_SMRwCCS = ["hydrogen production, steam methane reforming, w/ CCS"]
choices_hydrogen_PEM = ["hydrogen production, PEM electrolysis, yellow"]
choices_hydrogen_plastic = ["hydrogen production, plastics gasification"]
choices_hydrogen_plasticCCS = ["hydrogen production, plastics gasification, w/ CCS"]

choices_hydrogen_market = [
    "market for hydrogen",
    "market for hydrogen, gaseous, low pressure"
]

choices_heat_H2 = ["heat from hydrogen"]
choices_heat_CH4wCCS = ["heat from methane, w/ CCS"]
choices_heat_CH4 = ["heat from methane"]

choices_ammonia = [
    "ammonia production, steam methane reforming",
    # "ammonia production, steam methane reforming, w/ CCS",
    "ammonia production, from nitrogen and hydrogen"
]

choices_ammonia_market = [
    "market for ammonia",
    "market for ammonia, anhydrous, liquid"
]



In [186]:
# Retrieve activities for each category and assign to appropriately named variables

# Biomethane upgrading
biomethane_activities_CS = pulpo_worker.retrieve_activities(activities=choices_biomethane_CS, locations=["RER", "Europe without Switzerland"])
biomethane_activities_CSwCCS = pulpo_worker.retrieve_activities(activities=choices_biomethane_CSwCCS, locations=["RER", "Europe without Switzerland"])
biomethane_activities_WS = pulpo_worker.retrieve_activities(activities=choices_biomethane_WS, locations=["RER", "Europe without Switzerland"])
biomethane_activities_WSwCCS = pulpo_worker.retrieve_activities(activities=choices_biomethane_WSwCCS, locations=["RER", "Europe without Switzerland"])

methane_market_activities = pulpo_worker.retrieve_activities(activities=choices_methane_market, locations=["RER", "Europe without Switzerland"])
# Hydrogen
hydrogen_activities_PEM = pulpo_worker.retrieve_activities(activities=choices_hydrogen_PEM, locations=["RER", "Europe without Switzerland"])
hydrogen_activities_plastic = pulpo_worker.retrieve_activities(activities=choices_hydrogen_plastic, locations=["RER", "Europe without Switzerland"])
hydrogen_activities_plasticCCS = pulpo_worker.retrieve_activities(activities=choices_hydrogen_plasticCCS, locations=["RER", "Europe without Switzerland"])
hydrogen_activities_SMR = pulpo_worker.retrieve_activities(activities=choices_hydrogen_SMR, locations=["RER", "Europe without Switzerland"])
hydrogen_activities_SMRwCCS = pulpo_worker.retrieve_activities(activities=choices_hydrogen_SMRwCCS, locations=["RER", "Europe without Switzerland"])
hydrogen_market_activities = pulpo_worker.retrieve_activities(activities=choices_hydrogen_market, locations=["RER", "Europe without Switzerland"])
# Heat
heat_activities_CH4 = pulpo_worker.retrieve_activities(activities=choices_heat_CH4, locations=["RER", "Europe without Switzerland"])
heat_activities_CH4wCCS = pulpo_worker.retrieve_activities(activities=choices_heat_CH4wCCS, locations=["RER", "Europe without Switzerland"])
heat_activities_H2 = pulpo_worker.retrieve_activities(activities=choices_heat_H2, locations=["RER", "Europe without Switzerland"])
# Ammonia
ammonia_activities = pulpo_worker.retrieve_activities(activities=choices_ammonia, locations=["RER", "Europe without Switzerland"])
ammonia_market_activities = pulpo_worker.retrieve_activities(activities=choices_ammonia_market, locations=["RER", "Europe without Switzerland"])

In [187]:
# Display the results
print("Biomethane Activities:", biomethane_activities_CS+biomethane_activities_CSwCCS+biomethane_activities_WS+biomethane_activities_WSwCCS)
print("Methane Market Activities:", methane_market_activities)
print("Hydrogen Activities:", hydrogen_activities_PEM + hydrogen_activities_plastic + hydrogen_activities_plasticCCS + hydrogen_activities_SMR + hydrogen_activities_SMRwCCS)
print("Hydrogen Market Activities:", hydrogen_market_activities)
print("Heat Activities:", heat_activities_CH4+heat_activities_CH4wCCS+heat_activities_H2)
print("Ammonia Activities:", ammonia_activities)
print("Ammonia Market Activities:", ammonia_market_activities)

Biomethane Activities: ['biogas upgrading to biomethane, chemical scrubbing' (cubic meter, RER, None), 'biogas upgrading to biomethane, chemical scrubbing w/ CCS' (cubic meter, RER, None), 'biogas upgrading to biomethane, water scrubbing' (cubic meter, RER, None), 'biogas upgrading to biomethane, water scrubbing w/ CCS' (cubic meter, RER, None)]
Methane Market Activities: ['market group for natural gas, high pressure' (cubic meter, Europe without Switzerland, None), 'market for bio methane' (cubic meter, RER, None)]
Hydrogen Activities: ['hydrogen production, PEM electrolysis, yellow' (kilogram, RER, None), 'hydrogen production, plastics gasification' (kilogram, RER, None), 'hydrogen production, plastics gasification, w/ CCS' (kilogram, RER, None), 'hydrogen production, steam methane reforming fg' (kilogram, RER, None), 'hydrogen production, steam methane reforming, w/ CCS' (kilogram, RER, None)]
Hydrogen Market Activities: ['market for hydrogen, gaseous, low pressure' (kilogram, RER, 

### Set constraints on choices

In [188]:
# Since we can not solve the problem with high demands we need to scale the constraints
demand_value = 3000e6

In [189]:
# Setting the manure biomgas availability
manure_biogas = 1.8e9

In [190]:
choices = {
    "ammonia": {x: 1e20 for x in ammonia_activities},
    "methane_market": {x: 1e20 for x in methane_market_activities},
    "hydrogen_market": {x: 1e20 for x in hydrogen_market_activities},
    "ammonia_market": {x: 1e20 for x in ammonia_market_activities},
}
choices["hydrogen"] = {
    hydrogen_activities_PEM[0] : .3*.1e9/demand_value, # 0.1 Mt H2 -- 30% of PEM to NH3
    hydrogen_activities_SMR[0] : 1e20, # SMR
    hydrogen_activities_SMRwCCS[0] : .3*.5e9/demand_value, # 0.5 Mt H2 -- 30% of SMR w CCS H2 for NH3
    hydrogen_activities_plastic[0] : .3*40e6/demand_value, # 40 kt H2 -- 30% of plastic gasification for NH3
    hydrogen_activities_plasticCCS[0] : .3*1e6/demand_value, # 10 kt H2 -- 30% of plastic gasification w CCS for NH3
}
choices["heat"] = {
    heat_activities_CH4[0]: 1e20,
    heat_activities_CH4wCCS[0]: .1*1.4e10/demand_value, # Assuming 10% of the heat currently needed for Ammonia produciton
    heat_activities_H2[0]:1*1.4e10/demand_value, # Assuming 10% of the heat currently needed for Ammonia produciton
}
choices["biomethane"] = {
    biomethane_activities_CS[0]: 1e20, # Assumingly the same potential as WS
    biomethane_activities_CSwCCS[0]: .1*.2*manure_biogas/demand_value, # Guess: the same as WS wCCS
    biomethane_activities_WS[0]: 1e20, # Assumingly the base case technology
    biomethane_activities_WSwCCS[0]: .1*.2*manure_biogas/demand_value #  20% (biomethane for NH3) of 10% (CCS for biomethane )of 1.8 mt (biomethane) for which is the max capacity 2030
}

### Demand

In [191]:
ammonia_market = pulpo_worker.retrieve_activities(activities="market for ammonia")

In [192]:
demand = {"ammonia_market": 1}  # 3000e6

### Add additional constraints

In [193]:
# manure_collection_act = pulpo_worker.retrieve_activities(activities='animal manure collection', locations=["RER", "Europe without Switzerland"])

In [194]:
anaerobic_digestion = pulpo_worker.retrieve_activities(activities='anaerobic digestion of animal manure, with biogenic carbon uptake', locations=["RER", "Europe without Switzerland"])

In [195]:
upper_bound = {
    # manure_collection_act[0]: 1e6/demand_value,# From (Liebetrau, 2021) assuming sustainable food systems
    anaerobic_digestion[0]:.2*1.8e9/demand_value # 20% of 1.8 mt which is the max capacity 2030
    } 

### Instantiate and Solve

In [196]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand, upper_limit=upper_bound)
# instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


In [197]:
# results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)
# options = {'NEOS_EMAIL':'b.j.p.m.haussling.lowgren@cml.leidenuniv.nl'}
# results_pyomo = pulpo_worker.solve(solver_name='baron', options=options)

In [198]:
# pulpo_worker.summarize_results(zeroes=True)

In [199]:
results_pyomo_highs = pulpo_worker.solve()

Running HiGHS 1.8.0 (git hash: eda5cbe): Copyright (c) 2024 HiGHS under MIT licence terms
Cols:       47102 lower bounds    less than or equal to       -1e+20 are treated as -Infinity
Cols:       47102 upper bounds greater than or equal to        1e+20 are treated as +Infinity
Rows:       23537 lower bounds    less than or equal to       -1e+20 are treated as -Infinity
Rows:       23549 upper bounds greater than or equal to        1e+20 are treated as +Infinity
optimal solution found:  1.2847673773958053


In [200]:
pulpo_worker.summarize_results(zeroes=True)

## Total Impact(s)

,Weight,Value
Method,,
"('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')",1,1.284767


## Choices Made

### ammonia

,Value,Capacity
Metadata,,
"ammonia production, from nitrogen and hydrogen | ammonia production, from nitrogen and hydrogen | RER",0.670029,1.000000e+20


...

### biomethane

,Value,Capacity
Metadata,,
"biogas upgrading to biomethane, chemical scrubbing | biomethane, 24 bar | RER",0.045041,1.000000e+20
"biogas upgrading to biomethane, chemical scrubbing w/ CCS | biomethane, 24 bar | RER",0.012000,1.200000e-02
"biogas upgrading to biomethane, water scrubbing w/ CCS | biomethane, 24 bar | RER",0.012000,1.200000e-02


## Constraints

### Constraints Upper

,Key,Metadata,Value,Limit
ID,,,,
23532,"(ammonia-reduced, a2610a42cd93afb46b13b2e95273e1f7_copy1)","anaerobic digestion of animal manure, with biogenic carbon uptake | biogas | RER",0.12,0.12


In [201]:
results_data = pulpo_worker.extract_results()